In [64]:
import pdb
import pandas as pd
import numpy as np
import os
from scipy.stats import gaussian_kde
from datetime import datetime
from faker import Faker
import random
np.random.seed(3101)
random_state = np.random.RandomState(3101)
fake = Faker()

# Importing data:
bank1 = archive  
bank2 = bank+marketing

In [65]:
# print(os.getcwd())
bank_df_train = pd.read_csv('./data/main/train.csv')
bank_df_test = pd.read_csv('./data/main/test.csv')

In [66]:
print(bank_df_train.head())
print(len(bank_df_train))

# bank_df_train_clean = bank_df_train.drop_duplicates(subset="CustomerId", keep="first")
# print(len(bank_df_train_clean))

   id  CustomerId         Surname  CreditScore Geography Gender  Age  Tenure  \
0   0    15674932  Okwudilichukwu          668    France   Male   33       3   
1   1    15749177   Okwudiliolisa          627    France   Male   33       1   
2   2    15694510           Hsueh          678    France   Male   40      10   
3   3    15741417             Kao          581    France   Male   34       2   
4   4    15766172       Chiemenam          716     Spain   Male   33       5   

   Balance  NumOfProducts  HasCrCard  IsActiveMember  EstimatedSalary  Exited  
0        0              2          1               0          181,450       0  
1        0              2          1               1           49,504       0  
2        0              2          1               0          184,867       0  
3  148,883              1          1               1           84,561       0  
4        0              2          1               1           15,069       0  
165034


In [67]:
print(bank_df_test.head())
len(bank_df_test)
duplicates = bank_df_test.duplicated(subset=['CustomerId'], keep=False)
print(duplicates)
#no "exited" column

       id  CustomerId    Surname  CreditScore Geography  Gender  Age  Tenure  \
0  165034    15773898   Lucchese          586    France  Female   23       2   
1  165035    15782418       Nott          683    France  Female   46       2   
2  165036    15807120         K?          656    France  Female   34       7   
3  165037    15808905  O'Donnell          681    France    Male   36       8   
4  165038    15607314    Higgins          752   Germany    Male   38      10   

   Balance  NumOfProducts  HasCrCard  IsActiveMember  EstimatedSalary  
0        0              2          0               1          160,977  
1        0              1          1               0           72,549  
2        0              2          1               0          138,882  
3        0              1          1               0          113,932  
4  121,264              1          1               0          139,431  
0          True
1          True
2          True
3          True
4          True
       

Data Cleaning

Removed CustomerId column in both test & train since it is an unused var in the original dataset
All other columns returns no duplicates

In [68]:
bank_df_train = bank_df_train.loc[:, bank_df_train.columns!='CustomerId'] # extract all columns except for CustomerId
print(bank_df_train[bank_df_train.duplicated() == True]) # check for duplicate columns 

bank_df_test = bank_df_test.loc[:, bank_df_test.columns!='CustomerId']

n_train = len(bank_df_train)
n_test = len(bank_df_test)

Empty DataFrame
Columns: [id, Surname, CreditScore, Geography, Gender, Age, Tenure, Balance, NumOfProducts, HasCrCard, IsActiveMember, EstimatedSalary, Exited]
Index: []


# Adding and populating features

### 13. Churn Date: The exact date the customer decided to exit.

In [69]:
start_date = "2023-01-01"
end_date = "2023-12-31"

random_dates = pd.date_range(start=start_date, end=end_date, freq='D')
random_dates_str = random_dates.strftime('%Y-%m-%d')
bank_df_train['ChurnDate'] = np.where(bank_df_train['Exited'] == 1, np.random.choice(random_dates_str) , np.nan) 
print(bank_df_train['ChurnDate'].head(10))

0           nan
1           nan
2           nan
3           nan
4           nan
5    2023-07-20
6           nan
7           nan
8           nan
9           nan
Name: ChurnDate, dtype: object


### 14. Sign Up Date: When did the customer create an account
has to be linked with churn date if any, and tenure. Tenure is usually rounded down, so 


In [70]:
def generate_signon_date(churn_date, tenure, exited):
    if exited == 0:
        churn_date = "2023-12-31"
    churn_date = pd.to_datetime(churn_date)
    max = churn_date - pd.DateOffset(years=tenure)
    min = churn_date - pd.DateOffset(years=(tenure + 1))
    random_dates = pd.date_range(min, max, freq='D').strftime('%Y-%m-%d')
    random_date = np.random.choice(random_dates)
    return random_date


bank_df_train['SignDate'] = bank_df_train.apply(lambda row: generate_signon_date(row['ChurnDate'], row['Tenure'], row['Exited']), axis = 1)
print(bank_df_train[['SignDate', 'ChurnDate', 'Tenure']].head(10))

bank_df_test['SignDate'] = bank_df_train.apply(lambda row: generate_signon_date("2023-12-31", row['Tenure'], 0), axis = 1)
# print(bank_df_train_clean.head(10))

     SignDate   ChurnDate  Tenure
0  2020-03-08         nan       3
1  2022-12-17         nan       1
2  2013-06-08         nan      10
3  2021-07-08         nan       2
4  2018-05-19         nan       5
5  2019-05-26  2023-07-20       4
6  2015-07-29         nan       8
7  2022-08-11         nan       1
8  2019-04-15         nan       4
9  2019-01-22         nan       4


### 15.Transaction History 1
Detailed transaction data offers insights into spending patterns and engagement. (Transaction frequency in the last 28 days)

In [71]:
transaction_df = pd.read_excel('./data/transaction_history.xlsx')

# print(transaction_df.head())
transaction_df['DATE'] = pd.to_datetime(transaction_df['DATE'])

grouped_df = transaction_df.groupby('Account No')

results = []
for group_name, group_data in grouped_df:
    # Get max and min dates for the group
    max_date = group_data['DATE'].max()
    min_date = max_date - pd.Timedelta(days=28)
    
    # Filter group data for transactions within the date range
    filtered_group = group_data[(group_data['DATE'] >= min_date) & (group_data['DATE'] <= max_date)]
    
    # Calculate transaction frequency and amount for the group
    transaction_frequency = len(filtered_group)
    transaction_amount = filtered_group['DEPOSIT AMT'].sum()
    
    # Append results to the list
    results.append({
        'Account No': group_name,
        'Total Transaction Amount': transaction_amount,
        'Transaction Frequency': transaction_frequency
    })

# Create DataFrame from results
result_df = pd.DataFrame(results)

result_df['Total Transaction Amount'] = result_df['Total Transaction Amount'].astype(int)
pd.options.display.float_format = '{:,.0f}'.format
# print(result_df['Total Transaction Amount'])

# print("Minimum date in the dataset:", min_date)
# print("Maximum date in the dataset:", max_date)

kde_freq = gaussian_kde(result_df['Transaction Frequency'])
kde_amnt = gaussian_kde(result_df['Total Transaction Amount'])
# print(result_df['Total Transaction Amount'])

fake_data_freq = kde_freq.resample(len(bank_df_train)).flatten()
fake_data_freq = np.round(fake_data_freq).astype(int)
bank_df_train['TransactionFreq'] = fake_data_freq
fake_data_freq = kde_freq.resample(len(bank_df_test)).flatten()
fake_data_freq = np.round(fake_data_freq).astype(int)
bank_df_test['TransactionFreq'] = fake_data_freq

# fake_data_amt = kde_amnt.resample(len(bank_df_train)).flatten()
# # fake_data_amt = np.round(fake_data_amt).astype(int)
# bank_df_train['TransactionAmnt'] = fake_data_amt
# fake_data_amt = kde_amnt.resample(len(bank_df_test)).flatten()
# # fake_data_amt = np.round(fake_data_amt).astype(int)
# bank_df_test['TransactionAmnt'] = fake_data_amt

# print(bank_df_train['TransactionAmnt'].head())


### 16. Transaction History 2
Detailed transaction data offers insights into spending patterns and engagement. (Transaction amount in the last 28 days)


In [72]:
# Import reference set
# Warning: File size slightly big
trans_hist_data = pd.read_excel('./data/transaction_history.xlsx')

trans_hist_data.dtypes
# trans_hist_data.describe()

# Extract the most recent 2 sets of 3 months assuming the most recent data is the most accurate
trans_1st_3mths = trans_hist_data[(trans_hist_data['DATE'] >= pd.to_datetime('2019-01')) & (trans_hist_data['DATE'] <= pd.to_datetime('2019-03'))]
trans_2ns_3mths = trans_hist_data[(trans_hist_data['DATE'] >= pd.to_datetime('2018-10')) & (trans_hist_data['DATE'] <= pd.to_datetime('2018-12'))]

trans_dist_data = trans_1st_3mths.groupby('Account No').agg({'VALUE DATE': 'size', 'WITHDRAWAL AMT':'sum', 'DEPOSIT AMT': 'sum'}).reset_index()
trans_dist_data['TOTAL AMT'] = -trans_dist_data['WITHDRAWAL AMT'] + trans_dist_data['DEPOSIT AMT']

# kde = gaussian_kde(trans_dist_data['VALUE DATE'])
# train['Transaction Freq'] = abs(kde.resample(n_train).flatten()).astype(int)

kde = gaussian_kde(trans_dist_data['TOTAL AMT'])
bank_df_train['Transaction Amt'] = kde.resample(n_train).flatten().astype(float)

### 17. Call support frequency
Higher calls might indicate issues and dissatisfaction, affecting churn. Past year. Past month 

In [73]:
support_freq = pd.read_csv("./data/support_frequency.csv")

kde = gaussian_kde(support_freq['no_of_cases'])
bank_df_train['call support frequency'] = abs(kde.resample(n_train).flatten()/12).astype(int)
print(bank_df_train.head())


   id         Surname  CreditScore Geography Gender  Age  Tenure  Balance  \
0   0  Okwudilichukwu          668    France   Male   33       3        0   
1   1   Okwudiliolisa          627    France   Male   33       1        0   
2   2           Hsueh          678    France   Male   40      10        0   
3   3             Kao          581    France   Male   34       2  148,883   
4   4       Chiemenam          716     Spain   Male   33       5        0   

   NumOfProducts  HasCrCard  IsActiveMember  EstimatedSalary  Exited  \
0              2          1               0          181,450       0   
1              2          1               1           49,504       0   
2              2          1               0          184,867       0   
3              1          1               1           84,561       0   
4              2          1               1           15,069       0   

  ChurnDate    SignDate  TransactionFreq  Transaction Amt  \
0       nan  2020-03-08               21   

### 18.Relationship Count
Reflects the breadth of the customer's relationship with the bank.
Shld be correlated to 6: No. of products   

In [74]:
relationship_count = pd.read_csv('./data/loan/credit_train.csv')
relationship_count['Current Loan Amount'].fillna(0, inplace=True) 
if np.any(np.isinf(relationship_count['Current Loan Amount'])):
    # Handle infinite values, such as replacing them with a large finite value
    relationship_count['Current Loan Amount'].replace([np.inf, -np.inf], np.finfo(np.float64).max, inplace=True)

kde = gaussian_kde(relationship_count['Current Loan Amount'])
bank_df_train['RelationshipCount'] = abs(kde.resample(n_train).flatten()/12).astype(int)

print(bank_df_train['RelationshipCount'].head())

0    8225300
1    8387237
2      37815
3     135496
4     117630
Name: RelationshipCount, dtype: int32


C:\Users\Ethan\AppData\Local\Temp\ipykernel_2624\4051543762.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  relationship_count['Current Loan Amount'].fillna(0, inplace=True)


### 19. Months Inactive
Indicates customer disengagement, potentially preceding churn. 

In [75]:
np.random.seed(3101)
bank_df_train['MonthsInactive'] = np.minimum(np.random.normal(0.5, 1.5, n_train), bank_df_train['Tenure'] * 12)
bank_df_train['MonthsInactive'] = np.maximum(bank_df_train['MonthsInactive'], 0)

print(bank_df_train['MonthsInactive'].head(20))

0    1
1    1
2    2
3    0
4    1
5    2
6    3
7    1
8    1
9    0
10   0
11   0
12   0
13   1
14   0
15   1
16   3
17   0
18   2
19   1
Name: MonthsInactive, dtype: float64


### 20. Net Promoter Score (NPS)
Measure of customer satisfaction and loyalty.
Scaled from 1 to 10 


In [76]:
nps = pd.read_csv('./data/NPS.csv')

nps_data = nps.groupby('Customer Name').agg({'NPS':'mean'}).reset_index()
kde = gaussian_kde(nps_data['NPS'])
bank_df_train['NPS'] = abs(kde.resample(n_train).flatten()).astype(int)

### 21. Education
Education level might influence financial behavior and churn.
Shld be correlated to 5. Acct Balance


In [77]:
education_marital = pd.read_csv("./data/education_marital.csv")

education = education_marital['Education_Level'].unique()

edu, counts = np.unique(education, return_counts=True)
value_to_index = {value: i for i, value in enumerate(edu)}
numerical_data = [value_to_index[value] for value in education]

kde = gaussian_kde(numerical_data)

x_values = np.unique(numerical_data)
pdf_values = kde(x_values)

pmf = pdf_values / np.sum(pdf_values)

resampled_indices = np.random.choice(x_values, size=n_train, p=pmf)

resampled_values = [edu[index] for index in resampled_indices]

bank_df_train['Education'] = resampled_values

print(bank_df_train.head())

   id         Surname  CreditScore Geography Gender  Age  Tenure  Balance  \
0   0  Okwudilichukwu          668    France   Male   33       3        0   
1   1   Okwudiliolisa          627    France   Male   33       1        0   
2   2           Hsueh          678    France   Male   40      10        0   
3   3             Kao          581    France   Male   34       2  148,883   
4   4       Chiemenam          716     Spain   Male   33       5        0   

   NumOfProducts  HasCrCard  ...  Exited  ChurnDate    SignDate  \
0              2          1  ...       0        nan  2020-03-08   
1              2          1  ...       0        nan  2022-12-17   
2              2          1  ...       0        nan  2013-06-08   
3              1          1  ...       0        nan  2021-07-08   
4              2          1  ...       0        nan  2018-05-19   

  TransactionFreq Transaction Amt  call support frequency  RelationshipCount  \
0              21     -14,455,448                     

### 22 Employment Status
Indicates financial stability, affecting churn likelihood. 
Shld be correlated to 5. Acct Balance & 21. Education  

In [78]:
employment = pd.read_csv("./data/employment.csv")

employment_data = employment['job'].unique()

employment_status, counts = np.unique(employment_data, return_counts=True)
value_to_index = {value: i for i, value in enumerate(employment_status)}
numerical_data = [value_to_index[value] for value in employment_data]

kde = gaussian_kde(numerical_data)

x_values = np.unique(numerical_data)
pdf_values = kde(x_values)
pmf = pdf_values / np.sum(pdf_values)

resampled_indices = np.random.choice(x_values, size=n_train, p=pmf)

resampled_values = [employment_status[index] for index in resampled_indices]

bank_df_train['Employment Status'] = resampled_values

print(bank_df_train.head())

   id         Surname  CreditScore Geography Gender  Age  Tenure  Balance  \
0   0  Okwudilichukwu          668    France   Male   33       3        0   
1   1   Okwudiliolisa          627    France   Male   33       1        0   
2   2           Hsueh          678    France   Male   40      10        0   
3   3             Kao          581    France   Male   34       2  148,883   
4   4       Chiemenam          716     Spain   Male   33       5        0   

   NumOfProducts  HasCrCard  ...  ChurnDate    SignDate  TransactionFreq  \
0              2          1  ...        nan  2020-03-08               21   
1              2          1  ...        nan  2022-12-17             -263   
2              2          1  ...        nan  2013-06-08             -308   
3              1          1  ...        nan  2021-07-08             -175   
4              2          1  ...        nan  2018-05-19              876   

  Transaction Amt call support frequency  RelationshipCount  MonthsInactive  \
0

### 23. Marital Status
Can impact financial decision-making and churn behavior. 

In [79]:
education_marital = pd.read_csv("./data/education_marital.csv")

marital_status = education_marital['Marital_Status'].unique()

marital, counts = np.unique(marital_status, return_counts=True)
value_to_index = {value: i for i, value in enumerate(marital)}
numerical_data = [value_to_index[value] for value in marital_status]

kde = gaussian_kde(numerical_data)

x_values = np.unique(numerical_data)
pdf_values = kde(x_values)

pmf = pdf_values / np.sum(pdf_values)

resampled_indices = np.random.choice(x_values, size=n_train, p=pmf)

resampled_values = [marital[index] for index in resampled_indices]

bank_df_train['Marital Status'] = resampled_values

print(bank_df_train.head())

   id         Surname  CreditScore Geography Gender  Age  Tenure  Balance  \
0   0  Okwudilichukwu          668    France   Male   33       3        0   
1   1   Okwudiliolisa          627    France   Male   33       1        0   
2   2           Hsueh          678    France   Male   40      10        0   
3   3             Kao          581    France   Male   34       2  148,883   
4   4       Chiemenam          716     Spain   Male   33       5        0   

   NumOfProducts  HasCrCard  ...    SignDate  TransactionFreq  \
0              2          1  ...  2020-03-08               21   
1              2          1  ...  2022-12-17             -263   
2              2          1  ...  2013-06-08             -308   
3              1          1  ...  2021-07-08             -175   
4              2          1  ...  2018-05-19              876   

   Transaction Amt call support frequency RelationshipCount  MonthsInactive  \
0      -14,455,448                     37           8225300        

### 24. Housing Status
Reflects stability and long-term commitment, influencing churn. 

In [80]:
housing = pd.read_csv("./data/housing.csv")

housing_status = housing['House_Ownership'].unique()

house, counts = np.unique(housing_status, return_counts=True)
value_to_index = {value: i for i, value in enumerate(house)}
numerical_data = [value_to_index[value] for value in housing_status]

kde = gaussian_kde(numerical_data)

x_values = np.unique(numerical_data)
pdf_values = kde(x_values)

pmf = pdf_values / np.sum(pdf_values)

resampled_indices = np.random.choice(x_values, size=n_train, p=pmf)

resampled_values = [house[index] for index in resampled_indices]

bank_df_train['Housing Status'] = resampled_values

print(bank_df_train.head())

   id         Surname  CreditScore Geography Gender  Age  Tenure  Balance  \
0   0  Okwudilichukwu          668    France   Male   33       3        0   
1   1   Okwudiliolisa          627    France   Male   33       1        0   
2   2           Hsueh          678    France   Male   40      10        0   
3   3             Kao          581    France   Male   34       2  148,883   
4   4       Chiemenam          716     Spain   Male   33       5        0   

   NumOfProducts  HasCrCard  ...  TransactionFreq  Transaction Amt  \
0              2          1  ...               21      -14,455,448   
1              2          1  ...             -263        2,122,559   
2              2          1  ...             -308          383,972   
3              1          1  ...             -175        1,778,977   
4              2          1  ...              876          882,196   

   call support frequency RelationshipCount MonthsInactive  NPS  \
0                      37           8225300      

### 25. Number of Dependents
Impacts financial priorities and risk tolerance, affecting churn.

In [81]:
dependants_df = pd.read_csv('./data/education+dependents+maritalstatus/BankChurners.csv')

kde = gaussian_kde(dependants_df['Dependent_count'])

bank_df_train['Dependants'] = np.maximum(kde.resample(len(bank_df_train)).flatten(), 0)
bank_df_train['Dependants'] = np.round(bank_df_train['Dependants']).astype(int)
bank_df_test['Dependants'] = np.maximum(kde.resample(len(bank_df_test)).flatten(), 0)
bank_df_test['Dependants'] = np.round(bank_df_test['Dependants']).astype(int)

print(bank_df_train.head())

   id         Surname  CreditScore Geography Gender  Age  Tenure  Balance  \
0   0  Okwudilichukwu          668    France   Male   33       3        0   
1   1   Okwudiliolisa          627    France   Male   33       1        0   
2   2           Hsueh          678    France   Male   40      10        0   
3   3             Kao          581    France   Male   34       2  148,883   
4   4       Chiemenam          716     Spain   Male   33       5        0   

   NumOfProducts  HasCrCard  ...  Transaction Amt  call support frequency  \
0              2          1  ...      -14,455,448                      37   
1              2          1  ...        2,122,559                      19   
2              2          1  ...          383,972                       6   
3              1          1  ...        1,778,977                      11   
4              2          1  ...          882,196                       2   

   RelationshipCount MonthsInactive NPS      Education  Employment Status 

### 26. Marketing Offers Accepted
Indicates responsiveness to incentives, affecting churn. Range 0-1, (percentage of marketing offers they accept, e.g.5 offer, 4 accepted, the value of the column 0.8)

In [82]:
marketing_df = pd.read_csv('./data/marketing offers/marketing_campaign.csv', sep=';')


marketing_df["MarketingOffersAcceptance"] = (marketing_df['AcceptedCmp1'] + marketing_df['AcceptedCmp2'] + marketing_df['AcceptedCmp3'] + marketing_df['AcceptedCmp4'] + marketing_df['AcceptedCmp5']) / 5
# print(marketing_df.head())

kde = gaussian_kde(marketing_df['MarketingOffersAcceptance'])

bank_df_train['MarketingOffersAcceptance'] = np.maximum(kde.resample(len(bank_df_train)).flatten(), 0)
bank_df_test['MarketingOffersAcceptance'] = np.maximum(kde.resample(len(bank_df_test)).flatten(), 0)

print(bank_df_train.head())

   id         Surname  CreditScore Geography Gender  Age  Tenure  Balance  \
0   0  Okwudilichukwu          668    France   Male   33       3        0   
1   1   Okwudiliolisa          627    France   Male   33       1        0   
2   2           Hsueh          678    France   Male   40      10        0   
3   3             Kao          581    France   Male   34       2  148,883   
4   4       Chiemenam          716     Spain   Male   33       5        0   

   NumOfProducts  HasCrCard  ...  call support frequency  RelationshipCount  \
0              2          1  ...                      37            8225300   
1              2          1  ...                      19            8387237   
2              2          1  ...                       6              37815   
3              1          1  ...                      11             135496   
4              2          1  ...                       2             117630   

   MonthsInactive NPS      Education  Employment Status  Marit

### 27. Channel Used for Transactions
Reflects preferred banking channels and engagement level. 
faker

In [83]:
transaction_channel_df = pd.read_csv('./data/main_payment_method/WA_Fn-UseC_-Telco-Customer-Churn.csv')

# print(transaction_channel_df.head())
methods = transaction_channel_df['PaymentMethod'].unique().tolist()
print(methods)
faked_data = [random.choice(methods) for _ in range(len(bank_df_train))]
faked_data_2 = [random.choice(methods) for _ in range(len(bank_df_test))]

bank_df_train['PaymentMethod'] = faked_data
bank_df_test['PaymentMethod'] = faked_data_2
print(bank_df_train.head())

['Electronic check', 'Mailed check', 'Bank transfer (automatic)', 'Credit card (automatic)']
   id         Surname  CreditScore Geography Gender  Age  Tenure  Balance  \
0   0  Okwudilichukwu          668    France   Male   33       3        0   
1   1   Okwudiliolisa          627    France   Male   33       1        0   
2   2           Hsueh          678    France   Male   40      10        0   
3   3             Kao          581    France   Male   34       2  148,883   
4   4       Chiemenam          716     Spain   Male   33       5        0   

   NumOfProducts  HasCrCard  ...  RelationshipCount  MonthsInactive  NPS  \
0              2          1  ...            8225300               1    7   
1              2          1  ...            8387237               1    2   
2              2          1  ...              37815               2    0   
3              1          1  ...             135496               0    9   
4              2          1  ...             117630             

### 28. Customer Satisfaction Surveys
Provides direct feedback on satisfaction levels, predicting churn. 

In [84]:
cust_satisfaction_df = pd.read_csv('./data/satisfaction score/Customer-Churn-Records.csv')

kde = gaussian_kde(cust_satisfaction_df['Satisfaction Score'])

fake_data = np.maximum(kde.resample(len(bank_df_train)).flatten(), 0)
fake_data = np.minimum(fake_data.flatten(), 5)
fake_data = np.round(fake_data).astype(int)

fake_data_2 = np.maximum(kde.resample(len(bank_df_test)).flatten(), 0)
fake_data_2 = np.minimum(fake_data_2.flatten(), 5)
fake_data_2 = np.round(fake_data_2).astype(int)

bank_df_train['CustomerSatisfaction'] = fake_data
bank_df_test['CustomerSatisfaction'] = fake_data_2
# print(len(bank_df_test))
# print(len(fake_data_2))

print(bank_df_train['CustomerSatisfaction'].head(10))

0    2
1    4
2    5
3    3
4    1
5    2
6    1
7    4
8    2
9    3
Name: CustomerSatisfaction, dtype: int32


### 29. Feature Satisfaction
Scale on 1 to 5


In [85]:
feature_df = pd.read_csv('./data/feature_and_support_satisfaction/Customer-survey-data.csv')
feature_df = feature_df.dropna()
kde = gaussian_kde(feature_df['How satisfied were you with your overall delivery experience at Ali?                    1-5 where 1 = extremely dissatisfied and 5 = extremely satisfied'])
bank_df_train['FeatureSatisfaction'] = kde.resample(len(bank_df_train)).flatten().astype(int)
bank_df_test['FeatureSatisfaction'] = kde.resample(len(bank_df_test)).flatten().astype(int)

print(bank_df_train['FeatureSatisfaction'].head())

0    4
1    5
2    5
3    4
4    3
Name: FeatureSatisfaction, dtype: int32


### 30. Support Satisfaction
Scale on 1 to 5


In [86]:
feature_df = pd.read_csv('./data/feature_and_support_satisfaction/Customer-survey-data.csv')
feature_df = feature_df.dropna()
kde = gaussian_kde(feature_df['How satisfied were you with the speed of delivery at Alis?                                1-5 where 1 = extremely dissatisfied and 5 = extremely satisfied'])
bank_df_train['SupportSatisfaction'] = kde.resample(len(bank_df_train)).flatten().astype(int)
bank_df_test['SupportSatisfaction'] = kde.resample(len(bank_df_test)).flatten().astype(int)

print(bank_df_train['SupportSatisfaction'].head())

0    2
1    4
2    2
3    2
4    3
Name: SupportSatisfaction, dtype: int32


### 31.Service Support Frequency (per mth)

In [87]:
support_freq = pd.read_csv("./data/support_frequency.csv")
kde = gaussian_kde(support_freq['no_of_cases'])
bank_df_train['service support frequency'] = abs(kde.resample(n_train).flatten()/12).astype(int)

### 32. Income Source
Indicates financial stability and potential churn risk. 

In [88]:
income_source_df = pd.read_csv('./data/income source/adult.csv')

# print(transaction_channel_df.head())
income_sourcs = income_source_df['workclass'].unique().tolist()
income_sourcs.remove('?')
print(income_sourcs)
faked_data = [random.choice(income_sourcs) for _ in range(len(bank_df_train))]
faked_data_2 = [random.choice(income_sourcs) for _ in range(len(bank_df_test))]

bank_df_train['IncomeSource'] = faked_data
bank_df_test['IncomeSource'] = faked_data_2
print(bank_df_train.head())

['Private', 'Local-gov', 'Self-emp-not-inc', 'Federal-gov', 'State-gov', 'Self-emp-inc', 'Without-pay', 'Never-worked']
   id         Surname  CreditScore Geography Gender  Age  Tenure  Balance  \
0   0  Okwudilichukwu          668    France   Male   33       3        0   
1   1   Okwudiliolisa          627    France   Male   33       1        0   
2   2           Hsueh          678    France   Male   40      10        0   
3   3             Kao          581    France   Male   34       2  148,883   
4   4       Chiemenam          716     Spain   Male   33       5        0   

   NumOfProducts  HasCrCard  ...  Marital Status  Housing Status  Dependants  \
0              2          1  ...         Unknown    norent_noown           3   
1              2          1  ...         Unknown           owned           4   
2              2          1  ...        Divorced    norent_noown           5   
3              1          1  ...          Single           owned           0   
4              2 

### 33. Credit Utilization
Reflects financial health and potential churn risk for credit customers.

In [89]:
credit_df = pd.read_csv('./data/credit utilization/BankChurners.csv')

kde = gaussian_kde(credit_df['Avg_Utilization_Ratio'])

fake_data = np.maximum(kde.resample(len(bank_df_train)).flatten(), 0)
fake_data = np.minimum(fake_data.flatten(), 1)
# fake_data = np.round(fake_data).astype(int)
fake_data_2 = np.maximum(kde.resample(len(bank_df_test)).flatten(), 0)
fake_data_2 = np.minimum(fake_data_2.flatten(), 1)

bank_df_train['CreditUtilization'] = fake_data
bank_df_test['CreditUtilization'] = fake_data_2

print(bank_df_train.head())

   id         Surname  CreditScore Geography Gender  Age  Tenure  Balance  \
0   0  Okwudilichukwu          668    France   Male   33       3        0   
1   1   Okwudiliolisa          627    France   Male   33       1        0   
2   2           Hsueh          678    France   Male   40      10        0   
3   3             Kao          581    France   Male   34       2  148,883   
4   4       Chiemenam          716     Spain   Male   33       5        0   

   NumOfProducts  HasCrCard  ...  Housing Status  Dependants  \
0              2          1  ...    norent_noown           3   
1              2          1  ...           owned           4   
2              2          1  ...    norent_noown           5   
3              1          1  ...           owned           0   
4              2          1  ...          rented           2   

   MarketingOffersAcceptance              PaymentMethod CustomerSatisfaction  \
0                          0  Bank transfer (automatic)                 

### 34. Response to Previous Retention Efforts
Records success or failure of previous retention efforts, guiding future strategies. 

In [90]:
retention_df = pd.read_csv('./data/retention/HR_comma_sep.csv')

kde = gaussian_kde(retention_df['promotion_last_5years'])

fake_data = np.maximum(kde.resample(len(bank_df_train)).flatten(), 0)
fake_data = np.minimum(fake_data.flatten(), 1)
fake_data = np.round(fake_data).astype(int)
fake_data_2 = np.maximum(kde.resample(len(bank_df_test)).flatten(), 0)
fake_data_2 = np.minimum(fake_data_2.flatten(), 1)
fake_data_2 = np.round(fake_data_2).astype(int)

bank_df_train['Retention'] = fake_data
bank_df_test['Retention'] = fake_data_2

print(bank_df_train.head())

   id         Surname  CreditScore Geography Gender  Age  Tenure  Balance  \
0   0  Okwudilichukwu          668    France   Male   33       3        0   
1   1   Okwudiliolisa          627    France   Male   33       1        0   
2   2           Hsueh          678    France   Male   40      10        0   
3   3             Kao          581    France   Male   34       2  148,883   
4   4       Chiemenam          716     Spain   Male   33       5        0   

   NumOfProducts  HasCrCard  ...  Dependants  MarketingOffersAcceptance  \
0              2          1  ...           3                          0   
1              2          1  ...           4                          0   
2              2          1  ...           5                          0   
3              1          1  ...           0                          1   
4              2          1  ...           2                          0   

               PaymentMethod CustomerSatisfaction FeatureSatisfaction  \
0  Bank trans

### 35. Change in behavior before n after
Average of percentage of increase/decrease (ranging from 0 - infinity, but most of the times it will be ard 0-2)

In [91]:
bank_df_train['ChanegInBehaviourMkt'] = np.random.normal(2, 0.25, len(bank_df_train))
bank_df_test['ChanegInBehaviourMkt']= np.random.normal(2, 0.25, len(bank_df_test))

print(bank_df_train.head())

   id         Surname  CreditScore Geography Gender  Age  Tenure  Balance  \
0   0  Okwudilichukwu          668    France   Male   33       3        0   
1   1   Okwudiliolisa          627    France   Male   33       1        0   
2   2           Hsueh          678    France   Male   40      10        0   
3   3             Kao          581    France   Male   34       2  148,883   
4   4       Chiemenam          716     Spain   Male   33       5        0   

   NumOfProducts  HasCrCard  ...  MarketingOffersAcceptance  \
0              2          1  ...                          0   
1              2          1  ...                          0   
2              2          1  ...                          0   
3              1          1  ...                          1   
4              2          1  ...                          0   

               PaymentMethod  CustomerSatisfaction FeatureSatisfaction  \
0  Bank transfer (automatic)                     2                   4   
1         

### 36. Change in behavior before n after for Support

In [92]:
bank_df_train['ChanegInBehaviourCust'] = np.random.normal(2, 0.25, len(bank_df_train))
bank_df_test['ChanegInBehaviourCust']= np.random.normal(2, 0.25, len(bank_df_test))

print(bank_df_train.head())

   id         Surname  CreditScore Geography Gender  Age  Tenure  Balance  \
0   0  Okwudilichukwu          668    France   Male   33       3        0   
1   1   Okwudiliolisa          627    France   Male   33       1        0   
2   2           Hsueh          678    France   Male   40      10        0   
3   3             Kao          581    France   Male   34       2  148,883   
4   4       Chiemenam          716     Spain   Male   33       5        0   

   NumOfProducts  HasCrCard  ...              PaymentMethod  \
0              2          1  ...  Bank transfer (automatic)   
1              2          1  ...               Mailed check   
2              2          1  ...  Bank transfer (automatic)   
3              1          1  ...  Bank transfer (automatic)   
4              2          1  ...           Electronic check   

   CustomerSatisfaction  FeatureSatisfaction SupportSatisfaction  \
0                     2                    4                   2   
1                     

### 37. Previous Lifecycle status 

In [96]:
life_cycles = ['Active', 'Dormant', 'Reactivated'] #everything but churned

bank_df_train['PrevLifecycle'] = bank_df_train.apply(lambda row: 'Churned' if row['Exited'] == 1 else np.random.choice(life_cycles), axis=1)

print(bank_df_train['PrevLifecycle'].head(10))

0         Active
1        Dormant
2        Dormant
3    Reactivated
4         Active
5        Churned
6        Dormant
7         Active
8    Reactivated
9         Active
Name: PrevLifecycle, dtype: object


### 38. Current Lifecycle status 

### 39. Customer Happiness Status 

### 40. Customer Personas

### 41. Price Sensitivity %

### 42. Feature Driven % 

### 43. Service Driven % 

### 44. Social Influencer

# EDA
1. covarience matrix
2. LDA
3. apriori
4. domain knowledge

### 1. Covariance matrix

In [ ]:
#code for covariance matrix

### 2. LDA

In [ ]:
#code for LDA

### 3. Aprior

In [ ]:
#code for apriori

### 4. Domain knowledge

In [ ]:
#domain knowledge....

# Model training:
1. Random forest
2. XGBoost
3. Logistic regression